In [1]:
import numpy as np
import pandas as pd
from itertools import product
import pulp

In [4]:
# baking off intent-to-treat vs. analysis of compliance
n = 5000

compliance_partitions = ['always_taker', 'complier', 'defier', 'never_taker']
response_partitions = ['always_better', 'helped', 'hurt', 'never_better']
partition_types = np.array(list(product(compliance_partitions, response_partitions)))

partition_probabilities = np.random.random(16)
partition_probabilities = partition_probabilities / partition_probabilities.sum()

assert partition_probabilities.sum() == 1

In [5]:
# Participant simulation

# drawing participant compliance and response behaviors according to the
# specified distribution
participant_partition = np.random.choice(range(len(partition_types)), n, p=partition_probabilities)
compliance_type, response_type = list(zip(*partition_types[participant_partition]))

# randomly assigning participants to Control and Treatment groups
assignments = np.array(['control', 'treatment'])
participant_assignment = assignments[np.random.randint(0, 2, n)]

# compiling all information into our dataframe
df = pd.DataFrame({'assignment': participant_assignment,
                   'compliance_type': compliance_type,
                   'response_type': response_type})

In [6]:
# Simulate outcomes

# Depending on assignment and compliance type, do you take the treatment?
df['took_treatment'] = (df.compliance_type == 'always_taker') \
                       | ( (df.compliance_type == 'complier') & (df.assignment == 'treatment')) \
                       | ( (df.compliance_type == 'defier') & (df.assignment == 'control'))

df.took_treatment = df.took_treatment.astype('int32')

# Depending on whether you took the treatment and your response_type, what happens to you?
df['good_outcome'] = (df.response_type == 'always_better') \
                     | ( (df.response_type == 'helped') & (df.took_treatment) )

df.good_outcome = df.good_outcome.astype('int32')

In [7]:
# Intent to treat analysis:

print(df[df.assignment == 'treatment'].good_outcome.mean())
print(df[df.assignment == 'control'].good_outcome.mean())

itt_ate = df[df.assignment == 'treatment'].good_outcome.mean() \
- df[df.assignment == 'control'].good_outcome.mean() 

print("ATE: %6.4f" % itt_ate)

0.44166666666666665
0.4717741935483871
ATE: -0.0301


In [8]:
# Causal analysis

# get all the probabilities we need: p(z,x,y) for each z,x,y combination
states = list(product(['treatment','control'],[0,1], [0,1]))

p_states = {f"{assignment}/{'treated' if took == 1 else 'untreated'}/{'good' if outcome ==1 else 'bad'}" : 
            ((df.assignment == assignment) 
               & (df.took_treatment == took) 
               & (df.good_outcome == outcome)).mean()
            for assignment, took, outcome in states
            }

In [9]:
p_states

{'treatment/untreated/bad': 0.196,
 'treatment/untreated/good': 0.0452,
 'treatment/treated/bad': 0.0854,
 'treatment/treated/good': 0.1774,
 'control/untreated/bad': 0.1732,
 'control/untreated/good': 0.0518,
 'control/treated/bad': 0.0888,
 'control/treated/good': 0.1822}

In [10]:
true_ate = (df.response_type == 'helped').mean() - (df.response_type == 'hurt').mean()

In [22]:
# linear programming problem

min_problem = pulp.LpProblem("min ATE", pulp.LpMinimize)
max_problem = pulp.LpProblem("max ATE", pulp.LpMaximize)

# our hidden variables are the the probability of being in one of the partitions
partition_names = ['/'.join([compliance, response]) for compliance, response in partition_types]
q = {partition: pulp.LpVariable(partition, 0, 1) for partition in partition_names}

In [23]:
# since our hidden vars are probabilities the sum of them should all be under 1
min_problem += sum([v for k,v in q.items()]) == 1

In [24]:
# there's a natural mapping from probabilities we see to the hidden variables we have
# we'll spell these out one by one.
# there are probably smarter ways to express this as a vector operation,
# but this is easier to understand
min_problem += q['never_taker/never_better'] + q['defier/never_better'] \
             + q['never_taker/helped'] + q['defier/helped'] == p_states['treatment/untreated/bad']

min_problem += q['never_taker/always_better'] + q['defier/always_better'] \
             + q['never_taker/hurt'] + q['defier/hurt'] == p_states['treatment/untreated/good']

min_problem += q['always_taker/never_better'] + q['complier/never_better'] \
             + q['always_taker/hurt'] + q['complier/hurt'] == p_states['treatment/treated/bad']

min_problem += q['always_taker/always_better'] + q['complier/always_better'] \
             + q['always_taker/helped'] + q['complier/helped'] == p_states['treatment/treated/good']

min_problem += q['never_taker/never_better'] + q['complier/never_better'] \
             + q['never_taker/helped'] + q['complier/helped'] == p_states['control/untreated/bad']

min_problem += q['never_taker/always_better'] + q['complier/never_better'] \
             + q['never_taker/hurt'] + q['complier/hurt'] == p_states['control/untreated/good']

min_problem += q['always_taker/never_better'] + q['defier/never_better'] \
             + q['always_taker/hurt'] + q['defier/hurt'] == p_states['control/treated/bad']

min_problem += q['always_taker/always_better'] + q['defier/always_better'] \
             + q['always_taker/helped'] + q['defier/helped'] == p_states['control/treated/good']

In [27]:
min_problem += q['complier/helped'] + q['defier/helped'] + q['always_taker/helped'] + q['never_taker/helped'] \
              - q['complier/hurt'] - q['defier/hurt'] - q['always_taker/hurt'] - q['never_taker/hurt']

/usr/local/lib/python3.6/site-packages/pulp/pulp.py:1349: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [28]:
min_problem

min ATE:
MINIMIZE
1*always_taker_helped + -1*always_taker_hurt + 1*complier_helped + -1*complier_hurt + 1*defier_helped + -1*defier_hurt + 1*never_taker_helped + -1*never_taker_hurt + 0
SUBJECT TO
_C1: always_taker_always_better + always_taker_helped + always_taker_hurt
 + always_taker_never_better + complier_always_better + complier_helped
 + complier_hurt + complier_never_better + defier_always_better
 + defier_helped + defier_hurt + defier_never_better
 + never_taker_always_better + never_taker_helped + never_taker_hurt
 + never_taker_never_better = 1

_C2: defier_helped + defier_never_better + never_taker_helped
 + never_taker_never_better = 0.196

_C3: defier_always_better + defier_hurt + never_taker_always_better
 + never_taker_hurt = 0.0452

_C4: always_taker_hurt + always_taker_never_better + complier_hurt
 + complier_never_better = 0.0854

_C5: always_taker_always_better + always_taker_helped + complier_always_better
 + complier_helped = 0.1774

_C6: complier_helped + complier